In [1]:
import threading

In [2]:
d = [{'nome':'aaa'},{'nome':'aab'},{'nome':'acca'}]

In [50]:
import os
import pickle
ELEICAO_PATH = 'eleicao.p'

def open_eleicoes():
    if (os.path.isfile(ELEICAO_PATH)):
        print('open_eleicoes, ja existe eleicao')
        with open(ELEICAO_PATH, 'rb') as fp:
            eleicoes = pickle.load(fp)
    else: eleicoes = []
    return eleicoes

def save_eleicoes(eleicoes):
    with open(ELEICAO_PATH, 'wb') as fp:
        pickle.dump(eleicoes, fp, protocol=pickle.HIGHEST_PROTOCOL)


def criarsessao(command):

    # pegando as eleicoes passadas
    eleicoes = open_eleicoes()
    separate = command.split(' ')
    # checar quantidade minima de argumentos

    if (len(separate) < 4):
        return_message = 'Menos de 3 argumentos foram recebidos, por favor refaca a operacao'
    elif (len(separate) > 7):
        return_message = 'Mais de 7 argumentos foram recebidos, por favor refaca a operacao'
    else:
        escolhas = []
        nome_escolhas = []
        for aux in separate[2:]:
            nome_escolhas.append(aux)
        escolhas = [{'id':a, 'nome':nome_escolhas[a], 'votos':[]} for a in range(len(nome_escolhas))]
        # print('escolhas:', escolhas)
        if(len(eleicoes)==0):
            print('len eleicoes == 0')
            eleicao = {'nome':separate[1],'escolhas':escolhas}
            eleicoes = [eleicao]
            return_message = 'Sessao criada com sucesso.'
        else:
            print('len eleicoes != 0')
            def check_name(name, list_dict):
                for a in list_dict:
                    if a['nome'] == name:
                        return 0
                return 1
            # se nao for vazio, checa se ja tem alguma sessao com o mesmo nome
            nome = separate[1]
            if not check_name(nome, eleicoes):
                return_message = 'O nome desta sessao ja existe, tente novamente com um novo nome'
            else:
                eleicoes.append({'nome':separate[1],'escolhas':escolhas})
                save_eleicoes(eleicoes)
                return_message = 'Sessao criada com sucesso.'
    save_eleicoes(eleicoes)
    # print('eleicoes: ', eleicoes)
    return return_message

In [53]:
criarsessao('criarsessao blos123555 bla blob')
a = open_eleicoes()

open_eleicoes, ja existe eleicao
escolhas: [{'id': 0, 'nome': 'bla', 'votos': []}, {'id': 1, 'nome': 'blob', 'votos': []}]
len eleicoes != 0
eleicoes:  [{'nome': 'blos', 'escolhas': [{'id': 0, 'nome': 'bla', 'votos': []}, {'id': 1, 'nome': 'blob', 'votos': []}]}, {'nome': 'blos123', 'escolhas': [{'id': 0, 'nome': 'bla', 'votos': []}, {'id': 1, 'nome': 'blob', 'votos': []}]}, {'nome': 'blos123555', 'escolhas': [{'id': 0, 'nome': 'bla', 'votos': []}, {'id': 1, 'nome': 'blob', 'votos': []}]}]
open_eleicoes, ja existe eleicao


In [38]:
a = open_eleicoes()
print(a)

Ja existe eleicao
[{'nome': 'blos', 'escolhas': [{'id': 0, 'nome': 'bla', 'votos': []}, {'id': 1, 'nome': 'blob', 'votos': []}]}]


In [49]:
a = []
a.append(1)
print(a)

[1]


In [43]:
os.path.isfile('data.p')

True

In [58]:
# def check_votar_msg(mensagem):
#     print()

# def check_id_votar():
#     print()

# def get_available_session():
#     eleicoes = open_eleicoes()
eleicoes = open_eleicoes()
eleicoes


open_eleicoes, ja existe eleicao


[{'nome': 'blos',
  'escolhas': [{'id': 0, 'nome': 'bla', 'votos': []},
   {'id': 1, 'nome': 'blob', 'votos': []}]},
 {'nome': 'blos123',
  'escolhas': [{'id': 0, 'nome': 'bla', 'votos': []},
   {'id': 1, 'nome': 'blob', 'votos': []}]},
 {'nome': 'blos123555',
  'escolhas': [{'id': 0, 'nome': 'bla', 'votos': []},
   {'id': 1, 'nome': 'blob', 'votos': []}]}]

In [71]:
def quem_votou(eleicao):
    result = []
    for escolha in eleicao['escolhas']:
        for voto in escolha['votos']:
            result.append(voto)
    return result

def executar_voto(mensagem, user_id):
    '''
    com tudo validado, pegaremos a mensagem e efetuaremos o voto
    '''
    eleicoes = open_eleicoes()
    nome = mensagem.split(' ')[1]
    id_escolha = mensagem.split(' ')[2]
    print(f'nome: {nome}, id_escolha: {id_escolha}')
    # Caso o usuário não tenha votado em nenhuma das escolhas de uma dada sessao, o voto será computado
    for eleicao in eleicoes:
        if(eleicao['nome']==nome and user_id not in quem_votou(eleicao)):
            # print('entrou')
            for escolha in eleicao['escolhas']:
                # print()
                if(escolha['id'] == int(id_escolha)): 
                    escolha['votos'].append(user_id)
                    save_eleicoes(eleicoes)
                    return 'Voto computado'
    save_eleicoes(eleicoes)
    return 'Voto não computado, houve algum erro'
                

def comando_votar(mensagem, user_id):
    



In [73]:
executar_voto('votar blos 1', 1)

open_eleicoes, ja existe eleicao
nome: blos, id_escolha: 1


'Voto não computado, houve algum erro'

In [70]:
eleicoes = open_eleicoes()
eleicoes

open_eleicoes, ja existe eleicao


[{'nome': 'blos',
  'escolhas': [{'id': 0, 'nome': 'bla', 'votos': []},
   {'id': 1, 'nome': 'blob', 'votos': []}]},
 {'nome': 'blos123',
  'escolhas': [{'id': 0, 'nome': 'bla', 'votos': []},
   {'id': 1, 'nome': 'blob', 'votos': []}]},
 {'nome': 'blos123555',
  'escolhas': [{'id': 0, 'nome': 'bla', 'votos': []},
   {'id': 1, 'nome': 'blob', 'votos': []}]}]